## 实现Vision Transformer

分为以下几个部分：
1. PatchEmbed 类：图像嵌入层，将输入图像分割为多个patch并进行线性投影。

2. Attention 类：自注意力机制层，用于计算每个patch之间的注意力权重。

3. MLP 类：前馈神经网络层，用于对每个patch进行特征变换。

4. Block 类：Transformer块，包含自注意力层和前馈神经网络层。

5. VisionTransformer 类：完整的Vision Transformer模型，包含多个Transformer块。

In [2]:
#导入必要的库
import torch
import torch.nn as nn
from collections import OrderedDict
from functools import partial

In [3]:
class PatchEmbed(nn.Module):
    """
    对2D图像作Patch Embedding操作
    """
    def __init__(self, img_size=224, patch_size=16, in_c=3, embed_dim=768, norm_layer=None):
        """
        初始化PatchEmbed层
        Args:
            img_size (int): 输入图像的大小，默认224
            patch_size (int): 每个patch的大小，默认16x16
            in_c (int): 输入图像的通道数，默认3
            embed_dim (int): 输出的patch embedding维度，默认768
            norm_layer (nn.Module, optional): 归一化层，默认None
        """
        super().__init__()
        img_size=(img_size, img_size)
        patch_size=(patch_size, patch_size)
        self.img_size = img_size
        self.patch_size = patch_size
        self.grid_size=(img_size[0]//patch_size[0], img_size[1]//patch_size[1])#计算每个patch的尺寸，图像的长，宽//patch_size的长，宽
        self.num_patches=self.grid_size[0]*self.grid_size[1]#计算patch的数量，长*宽
        #定义卷积层
        self.proj=nn.Conv2d(in_c, embed_dim, kernel_size=patch_size, stride=patch_size)
        self.in_c = in_c
        self.embed_dim = embed_dim
        self.norm_layer = norm_layer(embed_dim) if norm_layer else nn.Identity()


    def forward(self,x):
        #x: [B, C, H, W]
        #out: [B, num_patches, embed_dim]
        #首先判断输入图像的尺寸是否符合要求
        B, C, H, W = x.shape
        assert H == self.img_size[0] and W == self.img_size[1], \
            f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]})."
        x=self.proj(x)#通过卷积层，将图像转换为patch embedding
        print(f"投影后的形状x.shape: {x.shape}")
        #将输出的维度从[B, embed_dim, grid_h, grid_w]转换为[B, num_patches, embed_dim]
        x=x.flatten(2).transpose(1, 2)
        print(f"转换后的形状x.shape: {x.shape}")
        #layernorm通常是针对每个patch的embedding进行归一化，而不是针对所有patch的embedding进行归一化
        x=self.norm_layer(x)
        
        return x

#简单测试
#创建一个PatchEmbed层
patch_embed=PatchEmbed(img_size=224, patch_size=16, in_c=3, embed_dim=768, norm_layer=nn.LayerNorm)
#创建一个随机输入张量
x=torch.randn(1, 3, 224, 224)
#前向传播
out=patch_embed(x)
print(f"输出形状out.shape: {out.shape}")


投影后的形状x.shape: torch.Size([1, 768, 14, 14])
转换后的形状x.shape: torch.Size([1, 196, 768])
输出形状out.shape: torch.Size([1, 196, 768])


# 多头自注意力机制 (Multi-Head Self-Attention)

## 1. 核心概念
**多头自注意力 (MSA)** 是 Transformer 架构的核心组件。它允许模型在处理序列（如图像块 Patch）中的每个元素时，能够同时关注序列中的其他所有位置，从而捕捉**全局上下文信息**。

与传统的卷积神经网络 (CNN) 关注局部特征不同，Attention 机制天生具有全局感受野。

## 2. 数学原理 (Scaled Dot-Product Attention)

标准的自注意力计算公式如下：

$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V
$$

其中：
*   **$Q$ (Query)**: 查询向量，代表当前元素正在寻找的信息。
*   **$K$ (Key)**: 键向量，代表被查询元素的索引信息。
*   **$V$ (Value)**: 值向量，代表被查询元素的实际内容信息。
*   **$\sqrt{d_k}$**: 缩放因子。用于防止 $QK^T$ 的点积结果过大，导致 Softmax 函数进入梯度极小的饱和区（梯度消失）。

## 3. 多头机制 (Multi-Head) 的作用

**“多头”** 意味着我们将 Query、Key 和 Value 向量分割成 $h$ 个独立的“头”（Head），并在不同的特征子空间中并行计算注意力。

*   **为什么要多头？**
    这类似于 CNN 中的多个卷积核（Filters）。不同的头可以学习关注不同的特征关系。例如，在图像中，一个头可能关注**形状轮廓**，另一个头可能关注**纹理颜色**，还有一个头可能关注**位置关系**。最后将这些不同的信息拼接起来，模型的表达能力更强。

*   **计算步骤**：
    1.  **线性投影**：通过全连接层将输入 $X$ 投影得到 $Q, K, V$。
    2.  **拆分 (Split)**：将总维度 `dim` 拆分为 `num_heads` $\times$ `head_dim`。
    3.  **并行计算**：对每个头独立计算 Attention Score。
    4.  **拼接 (Concat)**：将所有头的输出拼接回原来的维度。
    5.  **融合 (Projection)**：通过一个全连接层融合不同头的信息。

## 4. 代码实现映射

对应于 `VisionTransformer.py` 中的 `Attention` 类关键步骤：

| 代码操作 | 数学含义 |
| :--- | :--- |
| `self.qkv(x)` | $W_q, W_k, W_v$ 线性投影生成 Q, K, V |
| `.reshape(..., heads, head_dim)` | 拆分为多头 |
| `(q @ k.transpose(-2, -1))` | 计算 $QK^T$ 相似度矩阵 |
| `* self.scale` | 除以 $\sqrt{d_k}$ 进行缩放 |
| `.softmax(dim=-1)` | 归一化得到注意力权重 |
| `attn @ v` | 加权求和得到该头的输出 |
| `self.proj(x)` | $W_o$ 输出线性层，融合多头信息 |

## 5. 维度变换图解

假设输入形状为 `[Batch, N, Dim]`：

1.  **Input**: `[B, N, C]`
2.  **QKV Projection**: `[B, N, 3*C]` $\rightarrow$ `[B, N, 3, Heads, Head_Dim]`
3.  **Transpose**: `[3, B, Heads, N, Head_Dim]` (分离出 Q, K, V)
4.  **Attention Map**: `Q @ K.T` $\rightarrow$ `[B, Heads, N, N]` (得到 $N \times N$ 的关系矩阵)
5.  **Weighted Value**: `Attn @ V` $\rightarrow$ `[B, Heads, N, Head_Dim]`
6.  **Output**: Reshape & Concat $\rightarrow$ `[B, N, C]`

In [9]:
#多头自注意力机制的实现
class Attention(nn.Module):
    def __init__(
        self, dim, num_heads=8, qkv_bias=False, 
        qk_scale=None, attn_drop=0., proj_drop=0.):
        '''
        此函数用于初始化相关参数
        :param dim: 输入token的维度
        :param num_heads: 注意力多头数量
        :param qkv_bias: 是否使用偏置，默认False
        :param qk_scale: 缩放因子
        :param attn_drop_ratio: 注意力的比例
        :param proj_drop_ratio: 投影的比例
        '''
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim // num_heads #每个头注意的维度
        self.scale = qk_scale or head_dim ** -0.5

        #qkv：一个全连接层同时计算Q、K、V
        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        #随机在计算后的注意力矩阵上进行dropout，防止过拟合
        self.attn_drop = nn.Dropout(attn_drop)
        #多头注意力拼接后，来统筹不同头的注意力结果的投影层
        self.proj = nn.Linear(dim, dim)
        #投影层的dropout
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x):
        '''
        此函数用于前向传播
        :param x: 输入的token序列，形状为(batch_size, seq_len, dim)
        :return: 输出的token序列，形状为(batch_size, seq_len, dim)
        '''
        B,N,C=x.shape
        #qkv:同时计算三个矩阵Q、K、V
        qkv=self.qkv(x) #[B,num_patches+1,3*total_dim]
        qkv=qkv.reshape(B,N,3,self.num_heads,C//self.num_heads)#[B,num_patches+1,3,head_dim,num_dim_per_heads]
        qkv=qkv.permute(2,0,3,1,4) #[3,B,num_heads,num_patches+1,head_dim]
        q,k,v=qkv[0],qkv[1],qkv[2] #[B,num_heads,num_patches+1,head_dim]
        # 打印 q 的前 3×3 子矩阵（即前 3 行、前 3 列）
        print(f"q 的前 3×3 内容:\n{q[..., :3, :3,1]}")
        # 打印 k 的前 3×3 子矩阵（即前 3 行、前 3 列）
        print(f"k 的前 3×3 内容:\n{k[..., :3, :3,1]}")
        # 打印 v 的前 3×3 子矩阵（即前 3 行、前 3 列）
        print(f"v 的前 3×3 内容:\n{v[..., :3, :3,1]}")

        #计算注意力矩阵
        # q: [B, H, N, D]
        # k: [B, H, N, D] -> k.transpose(-2, -1): [B, H, D, N],倒数第一个和倒数第二个维度互换位置
        # @ 是矩阵乘法
        print(f"q.shape: {q.shape}")
        print(f"k.transpose(-2, -1).shape: {k.transpose(-2, -1).shape}")
        attn=(q@k.transpose(-2,-1))*self.scale
        print(f"attn 的前 3×3 内容:\n{attn[..., :3, :3,1]}")
        #对注意力矩阵进行softmax归一化,沿着最后一个维度归一化,将原始的得分转换成概率分布
        attn=attn.softmax(dim=-1)
        print(f"经过归一化后的attn 的前 3×3 内容:\n{attn[..., :3, :3,1]}")
        #进行drop
        attn=self.attn_drop(attn)

        #加权求和
        # attn: [B, H, N, N]
        # v:    [B, H, N, D]
        # @ 是矩阵乘法
        x=(attn@v).transpose(1,2).reshape(B,N,C) 
        print(f"attn@v 的前 3×3 内容:\n{(attn@v)[..., :3, :3,1]}")
        print(f"x的内容: {x}")
        #[B,num_patches+1,num_heads,head_dim] -> [B,num_patches+1,total_dim]
        #投影层
        x=self.proj(x)
        print(f"投影层后的x 的前 3×3 内容:\n{x[..., :3, :3,1]}")
        #dropout
        x=self.proj_drop(x)
        return x

#简单测试
x=torch.randn(1,10,512)
attn=Attention(512)
print(attn(x).shape)



q 的前 3×3 内容:
tensor([[[-0.5844,  0.1977, -0.2092],
         [-0.2171, -0.1312, -1.0171],
         [-0.4126,  0.4842,  0.9619]]], grad_fn=<SelectBackward0>)
k 的前 3×3 内容:
tensor([[[-1.3133, -0.0220,  0.4310],
         [-0.1495, -0.6505, -0.3297],
         [-0.6323, -0.4088,  0.7506]]], grad_fn=<SelectBackward0>)
v 的前 3×3 内容:
tensor([[[ 0.6186, -0.6955,  0.0465],
         [-0.4180, -0.7046,  0.1503],
         [ 0.0653,  0.3319, -0.6398]]], grad_fn=<SelectBackward0>)
q.shape: torch.Size([1, 8, 10, 64])
k.transpose(-2, -1).shape: torch.Size([1, 8, 64, 10])
attn 的前 3×3 内容:
tensor([[[ 0.0107,  0.5636, -0.2711],
         [-0.0312, -0.6355, -0.1421],
         [ 0.5742,  0.3488, -0.2603]]], grad_fn=<SelectBackward0>)
经过归一化后的attn 的前 3×3 内容:
tensor([[[0.1215, 0.1237, 0.0735],
         [0.0953, 0.0509, 0.0656],
         [0.1394, 0.1321, 0.0772]]], grad_fn=<SelectBackward0>)
attn@v 的前 3×3 内容:
tensor([[[ 0.0151, -0.1505, -0.0181],
         [ 0.0109, -0.0081,  0.0405],
         [ 0.0816,  0.0406,  0.1